In [ ]:
import neo4j
import pandas
import pyspark
import random
import requests

In [ ]:
neo4j_url = "neo4j://neo4j:7687"
neo4j_user = "neo4j"
neo4j_password = "neo4jtp9"

URI = neo4j_url
AUTH = (neo4j_user, neo4j_password)

with neo4j.GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [ ]:
spark_session = (pyspark.sql.SparkSession.builder
    .master('spark://spark:7077')
    .appName('Data engineering workflow with Neo4j and Spark')
    .config('spark.ui.port', '4050')
    .config('spark.sql.repl.eagerEval.enabled', True)
    .config('spark.jars.packages', 'org.neo4j:neo4j-connector-apache-spark_2.12:5.1.0_for_spark_3,graphframes:graphframes:0.8.4-spark3.5-s_2.12')
    .config("neo4j.url", neo4j_url)
    .config("neo4j.authentication.type", "basic")
    .config("neo4j.authentication.scheme", "basic")
    .config("neo4j.authentication.basic.username", neo4j_user)
    .config("neo4j.authentication.basic.password", neo4j_password)
    .getOrCreate())

In [ ]:
arrets_information = requests.get("https://open.tan.fr/ewp/arrets.json")

In [ ]:
arrets = []
lignes = set()
relations = []
if arrets_information.status_code == 200 :
    data = arrets_information.json()
    for arret in data :
        arrets.append({"codeLieu": arret["codeLieu"], "libelle": arret["libelle"]})
        for l in arret["ligne"] :
            lignes.add(l["numLigne"])
            relations.append((arret["codeLieu"], l["numLigne"]))

nb_arrets =len(arrets)
nb_lignes =len(lignes)
nb_relations =len(relations)

In [ ]:
SEUIL = 30

lignes_to_remove = []
for ligne in lignes :
    taille = len(list(filter(lambda x: x[1] == ligne, relations)))
    if taille < SEUIL :
        lignes_to_remove.append(ligne)

for ligne in lignes_to_remove :
    lignes.remove(ligne)

relations = list(filter(lambda x: x[1] in lignes, relations))

arrets_to_keep = set()
for relation in relations :
    arrets_to_keep.add(relation[0])
arrets = list(filter(lambda x: x["codeLieu"] in arrets_to_keep, arrets))

nb_arrets_2 = len(arrets)
nb_lignes_2 = len(lignes)
nb_relations_2 = len(relations)

print("Arret : ", nb_arrets)
print("Ligne : ", nb_lignes)
print("Relation : ", nb_relations)
print("==========")
print("Arret : ", nb_arrets_2)
print("Ligne : ", nb_lignes_2)
print("Relation : ", nb_relations_2)

In [ ]:
driver = neo4j.GraphDatabase.driver(URI, auth=AUTH)
session = driver.session()

In [ ]:
driver.execute_query("MATCH (a)-[r]->() DELETE a, r")
driver.execute_query("MATCH (a) DELETE a")

for arret in arrets :
    driver.execute_query("CREATE (%s:Arret {codeArret:'%s' ,libelle:'%s'})"%(arret["codeLieu"], arret["codeLieu"], arret["libelle"].replace("'", " ")))

for ligne in lignes :
    driver.execute_query("CREATE (L_%s:Ligne {numLigne:'%s'})"%(ligne, ligne))

for relation in relations :
    driver.execute_query("MATCH (a:Arret {codeArret: '%s'}), (b:Ligne {numLigne: '%s'}) MERGE (a)-[:IN]->(b)"%(relation[0], relation[1]))
    driver.execute_query("MATCH (a:Arret {codeArret: '%s'}), (b:Ligne {numLigne: '%s'}) MERGE (b)-[:IN]->(a)"%(relation[0], relation[1]))

In [ ]:
df_arrets = (spark_session.read
    .format('org.neo4j.spark.DataSource')
    .option('labels', ':Arret')
    .load())

df_arrets.show()

In [ ]:
df_lignes = (spark_session.read
    .format('org.neo4j.spark.DataSource')
    .option('labels', ':Ligne')
    .load())

df_lignes.show()

In [ ]:
df_relations = (spark_session.read
    .format('org.neo4j.spark.DataSource')
    .option('relationship', 'IN')
    .option('relationship.source.labels', ':Arret')
    .option('relationship.target.labels', ':Ligne')
    .load())

df_relations = df_relations.withColumnsRenamed({
    "<rel.id>":"id",
    "<rel.type>":"relationship",
    "<source.id>":"src",
    "<source.labels>":"source_labels",
    "source.libelle":"source_libelle",
    "source.codeArret":"source_codeArret",
    "<target.id>":"dst",
    "<target.labels>":"target_labels",
    "target.numLigne":"target_numLigne"
})

df_relations.show()

In [ ]:
df_arrets_id = df_arrets.withColumnsRenamed({"<id>": "id", "<labels>": "labels", "codeArret": "name"}).select("id", "labels", "name")
df_lignes_id = df_lignes.withColumnsRenamed({"<id>": "id", "<labels>": "labels", "numLigne": "name"}).select("id", "labels", "name")
df_arrets_et_lignes = df_arrets_id.union(df_lignes_id)

df_arrets_et_lignes.show()

In [ ]:
records, summary, keys = driver.execute_query(
    """
    MATCH   (a1:Arret)-[:IN]->(l1:Ligne),
            (a2:Arret)-[:IN]->(l1)
    WHERE   a1.codeArret = 'CTRE'
    ORDER BY l1.numLigne, a2.codeArret
    RETURN DISTINCT a1.codeArret, l1.numLigne, a2.codeArret
    """,
)

pandas.DataFrame(records, columns=keys)

In [ ]:
records, summary, keys = driver.execute_query(
    """
    MATCH   p = ALL SHORTEST (a1:Arret)-[:IN]-+(a2:Arret)
    WHERE   a1.codeArret = 'CTRE' AND a2.codeArret = 'INRE'
    RETURN  DISTINCT [n in nodes(p) WHERE n:Arret | n.codeArret] AS stops
    """
)

pandas.DataFrame(records, columns=keys)

In [ ]:
pandas_df = driver.execute_query(
    """
    MATCH   p = ALL SHORTEST (a1:Arret)-[:IN]-+(a2:Arret)
    WHERE   a1.codeArret = 'CTRE' AND a2.codeArret = 'BCGA'
    RETURN  DISTINCT [n in nodes(p) WHERE n:Arret | n.codeArret] AS stops
    """,
    result_transformer_ = neo4j.Result.to_df
)

pandas_df

In [ ]:
random_source = random.choice(arrets)
random_target = random.choice(arrets)

print("Random Source : ", random_source)
print("Random Target : ", random_target)

pandas_df = driver.execute_query(
    """
    MATCH   p = ALL SHORTEST (a1:Arret)-[:IN]-+(a2:Arret)
    WHERE   a1.codeArret = '%s' AND a2.codeArret = '%s'
    RETURN  DISTINCT [n in nodes(p) WHERE n:Arret | n.codeArret] AS stops
    ORDER BY stops
    """%(random_source["codeLieu"], random_target["codeLieu"]),
    result_transformer_ = neo4j.Result.to_df
)

pandas_df